# Reading and writing files

Vector data cubes do not have an official specification for an interoperable file format that can be used to save any Xarray object with geometric dimensions to a file and read it back or in other software like [`{stars}`](https://r-spatial.github.io/stars/) in the R-Spatial ecosystem.

However, there are some ways to encode the data to be compliant with either array I/O. The CF conventions offer a way of encoding coordinate geometries to array data that can then be saved to an array format like [netCDF](https://www.unidata.ucar.edu/software/netcdf/) or [Zarr](https://zarr.readthedocs.io/). Both coordinate geometry and variable geometry can be encoded as well-known binary representation and stored in Zarr, with additional attributes allowing for lossless rountripping through a file. Both of these are supported by Xvec.

First lets read some data.

In [19]:
# !pip install git+https://github.com/martinfleis/xvec.git@summary

In [20]:
import geopandas as gpd
import xarray as xr
from geodatasets import get_path
import xproj
import zarr

import xvec

In [21]:
counties = gpd.read_file(get_path("geoda.natregimes"))

cube = xr.Dataset(
    data_vars=dict(
        population=(["county", "year"], counties[["PO60", "PO70", "PO80", "PO90"]]),
        unemployment=(["county", "year"], counties[["UE60", "UE70", "UE80", "UE90"]]),
        divorce=(["county", "year"], counties[["DV60", "DV70", "DV80", "DV90"]]),
        age=(["county", "year"], counties[["MA60", "MA70", "MA80", "MA90"]]),
    ),
    coords=dict(county=counties.geometry, year=[1960, 1970, 1980, 1990]),
).xvec.set_geom_indexes("county", crs=counties.crs)
cube

<xarray.Dataset> Size: 370kB
Dimensions:       (county: 3085, year: 4)
Coordinates:
  * county        (county) object 25kB POLYGON ((-95.34258270263672 48.546703...
  * year          (year) int64 32B 1960 1970 1980 1990
Data variables:
    population    (county, year) int32 49kB 4304 3987 3764 ... 43766 55800 65077
    unemployment  (county, year) float64 99kB 7.9 9.0 5.903 ... 7.018 5.489
    divorce       (county, year) float64 99kB 1.859 2.62 3.747 ... 4.782 7.415
    age           (county, year) float64 99kB 28.8 30.5 34.5 ... 28.97 35.33
Indexes:
    county   GeometryIndex (crs=EPSG:4326)

## CF conventions and netCDF, Zarr

Given a vector data cube that contains only coordinate geometries (i.e. no variable geometries), you can use `.xvec.encode_cf()` to to encode geometry arrays with CF conventions to a form that is compatible with any array storage format (e.g. netCDF, Zarr). This function uses `cf_xarray.geometry.encode_geometries` and `cf_xarray.geometry.decode_geometries` under the hood and is not limited to a single array of geometries.


In [22]:
encoded = cube.xvec.encode_cf()
encoded

<xarray.Dataset> Size: 2MB
Dimensions:             (county: 3085, part: 3172, node: 80563, year: 4)
Coordinates:
    spatial_ref         int64 8B 0
  * year                (year) int64 32B 1960 1970 1980 1990
Dimensions without coordinates: county, part, node
Data variables:
    node_count          (county) int64 25kB 24 50 68 99 16 7 ... 86 32 24 32 74
    part_node_count     (part) int64 25kB 24 50 68 99 16 7 ... 28 86 32 24 32 74
    interior_ring       (part) bool 3kB False False False ... False False False
    geometry_container  float64 8B nan
    x                   (node) float64 645kB -95.34 -95.34 ... -111.3 -111.4
    y                   (node) float64 645kB 48.55 48.72 48.72 ... 44.73 44.75
    lon                 (county) float64 25kB -95.34 -118.9 ... -77.53 -111.4
    lat                 (county) float64 25kB 48.55 47.95 48.04 ... 38.57 44.75
    population          (county, year) int32 49kB 4304 3987 3764 ... 55800 65077
    unemployment        (county, year) float64 99kB 7.9 9.0 ... 7.018 5.489
    divorce             (county, year) float64 99kB 1.859 2.62 ... 4.782 7.415
    age                 (county, year) float64 99kB 28.8 30.5 ... 28.97 35.33

Write to a format as usual with Xarray

In [23]:
encoded.to_netcdf("../../data/raw/vdc-examples/geo-encoded-xvec.nc", mode="w")
encoded.to_zarr("../../data/raw/vdc-examples/geo-encoded-xvec.zarr", mode="w")

/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at node_count is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at age is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at lon is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at geometry_container is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at part_node_count is not recognized as a component of a Zarr hierarchy.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/zarr/core/group.py:2824: UserWarning: Object at interior_ring is not recognized as a component of a Zarr hierar

On open, use `.xvec.decode_cf` to recover the array. This function uses `cf_xarray.decode_geometries` so again `cf_xarray` must be installed.

In [24]:
xr.open_zarr("../../data/raw/vdc-examples/geo-encoded-xvec.zarr").xvec.decode_cf()

<xarray.Dataset> Size: 370kB
Dimensions:       (county: 3085, year: 4)
Coordinates:
  * county        (county) object 25kB POLYGON ((-95.34258270263672 48.546703...
  * year          (year) int64 32B 1960 1970 1980 1990
Data variables:
    age           (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    population    (county, year) int32 49kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    divorce       (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    unemployment  (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
Indexes:
    county   GeometryIndex (crs=EPSG:4326)

In [25]:
roundtripped = xr.open_zarr("geo-encoded.zarr").xvec.decode_cf()
roundtripped

<xarray.Dataset> Size: 370kB
Dimensions:       (county: 3085, year: 4)
Coordinates:
  * county        (county) object 25kB POLYGON ((-95.34258270263672 48.546703...
  * year          (year) int64 32B 1960 1970 1980 1990
Data variables:
    divorce       (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    age           (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    population    (county, year) int32 49kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
    unemployment  (county, year) float64 99kB dask.array<chunksize=(3085, 4), meta=np.ndarray>
Indexes:
    county   GeometryIndex (crs=EPSG:4326)

In [26]:
roundtripped.identical(cube)

True

## Read files created with `stars::write_mdim()`

### NetCDF

Can be read but it is not decoded correctly. Note that the stations are floats and the geometries are placed in a data variable with the name None.

In [27]:
mdim = xr.open_dataset("../../data/raw/vdc-examples/mdim-stars.nc", engine="netcdf4").xvec.decode_cf()
mdim

<xarray.Dataset> Size: 104B
Dimensions:        (features: 2, time: 5, stations: 2)
Coordinates:
  * stations       (stations) float32 8B 1.0 2.0
  * time           (time) datetime64[ns] 40B 2022-05-02 ... 2022-05-06
Dimensions without coordinates: features
Data variables:
    None           (features) object 16B POINT (0 1) POINT (3 5)
    Precipitation  (time, stations) float32 40B ...

In [28]:
mdim_encoded = mdim.xvec.encode_cf()
mdim_encoded

<xarray.Dataset> Size: 160B
Dimensions:             (node: 2, features: 2, time: 5, stations: 2)
Coordinates:
  * stations            (stations) float32 8B 1.0 2.0
  * time                (time) datetime64[ns] 40B 2022-05-02 ... 2022-05-06
Dimensions without coordinates: node, features
Data variables:
    geometry_container  float64 8B nan
    x                   (node) float64 16B 0.0 3.0
    y                   (node) float64 16B 1.0 5.0
    crd_x               (features) float64 16B 0.0 3.0
    crd_y               (features) float64 16B 1.0 5.0
    Precipitation       (time, stations) float32 40B ...

In [29]:
mdim_encoded.to_netcdf("../../data/raw/vdc-examples/mdim-xvec.nc", mode="w")

TypeError: Invalid value for attr 'variable_name': None. For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple, bytes

### Zarr

In [30]:
xr.open_zarr("../../data/raw/vdc-examples/mdim-stars.zarr").xvec.decode_cf()

TypeError: 'LocalStore' object is not subscriptable

## Files encoded with xvec, encoded and decoded with stars, and again decoded with xvec

In [31]:
xr.open_dataset("../../data/raw/vdc-examples/geo-encoded-stars.nc", engine="netcdf4").xvec.decode_cf()

TypeError: Cannot cast array data from dtype('float32') to dtype('int64') according to the rule 'safe'

In [32]:
xr.open_zarr("../../data/raw/vdc-examples/geo-encoded-stars.zarr").xvec.decode_cf()

TypeError: 'LocalStore' object is not subscriptable